Librerías

In [ ]:
import pandas as pd
import numpy as np

import os
import joblib

import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

from sklearn.metrics import mean_absolute_error, mean_squared_error

from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.graph_objects as go

- Ce carga un subdataset con los datos necesarios para el modelo de ML

In [ ]:
# Cargar los datos
df = pd.read_csv(r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Input.csv")
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Obtener el valor máximo de la columna 'date'
max_date = df['date'].max()
# Calcular la fecha de diciembre dentro de 5 años
future_date = datetime(max_date.year + 5, 12, 1)
# Calcular la cantidad de meses entre ambas fechas
months_difference = (future_date.year - max_date.year) * 12 + (future_date.month - max_date.month)

In [ ]:
# Función para guardar el forecast en un archivo CSV
def guardar_datos_forecast(forecast, industry_type, column_name, output_file=r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output.csv"):
    """
    Guarda los resultados del pronóstico en un archivo CSV consolidado.
    Si ya existe un pronóstico para la combinación `industry_type` y `column_name`, no lo vuelve a guardar.
    """
    # Crear un DataFrame con la información del pronóstico
    forecast_data = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    forecast_data['industry'] = industry_type
    forecast_data['column'] = column_name

    # Verificar si el archivo existe
    if os.path.exists(output_file):
        # Cargar datos existentes
        existing_data = pd.read_csv(output_file)
        
        # Filtrar para ver si ya existe esta combinación
        exists = (
            (existing_data['industry'] == industry_type) &
            (existing_data['column'] == column_name)
        ).any()

        if exists:
            print(f"Pronóstico para la combinación '{industry_type}' - '{column_name}' ya existe en {output_file}.")
            return

        # Concatenar datos nuevos si no existen
        combined_data = pd.concat([existing_data, forecast_data], ignore_index=True)
    else:
        # Crear un nuevo archivo si no existe
        combined_data = forecast_data

    # Guardar el archivo consolidado
    combined_data.to_csv(output_file, index=False)
    print(f"Pronóstico guardado en {output_file}.")


In [ ]:
def graficar_original(df_prophet, column_name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], marker=dict(symbol='circle', color='royalblue')))
    fig.layout.update(title_text="Datos históricos", yaxis_title=f"{column_name}", xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
def graficar_predicción(df_prophet, column_name, forecast,industry_type):
    """
    Función para graficar la predicción realizada por Prophet junto con el intervalo de confianza.
    Ajusta automáticamente el eje Y al rango de valores de los datos.
    """
    # Crear figura
    fig = go.Figure()

    # Datos históricos
    fig.add_trace(go.Scatter( x=df_prophet["ds"], y=df_prophet["y"], mode='lines+markers', name='Datos Históricos', marker=dict(symbol='circle', color='royalblue'),
        line=dict(color='royalblue', width=2)))

    # Predicción
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicción', line=dict(color='green', width=3, dash='dot')))

    # Intervalo de predicción
    fig.add_trace(go.Scatter( x=forecast['ds'].tolist() + forecast['ds'][::-1].tolist(), y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'][::-1].tolist(), 
        fill='toself', fillcolor='rgba(0, 128, 0, 0.2)', line=dict(color='rgba(0, 0, 0, 0)'), name='Intervalo de Predicción' ))

    # Calcular el rango automático para el eje Y
    y_values = (df_prophet["y"].tolist() + forecast['yhat'].tolist() ) + forecast['yhat_upper'].tolist() #+ forecast['yhat_lower'].tolist() 
    
    y_min, y_max = min(y_values), max(y_values)

    # Configuración del layout
    fig.update_layout( title=f"Predicción para {column_name} ({industry_type})", xaxis_title='Fecha', yaxis_title=column_name, xaxis_rangeslider_visible=True,
        yaxis=dict(range=[y_min, y_max]), template='plotly_white',
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
    )

    # Mostrar gráfico
    fig.show()    

In [ ]:
# Función para obtener el nombre del archivo del modelo
def obtener_nombre_archivo(industry_type, column_name):
    """Genera un nombre de archivo único para una combinación de industria y columna."""
    return f"models/prophet_model_{industry_type}_{column_name}.pkl"

# Función para cargar o entrenar un modelo Prophet
def cargar_o_entrenar_modelo(df_prophet, industry_type, column_name, cps, sps, sm):
    """
    Carga un modelo Prophet guardado o lo entrena y guarda si no existe.
    """
    # Crear el directorio "models" si no existe
    os.makedirs("models", exist_ok=True)
    
    # Nombre del archivo del modelo
    modelo_path = obtener_nombre_archivo(industry_type, column_name)
    
    # Intentar cargar el modelo si ya existe
    if os.path.exists(modelo_path):
        print(f"Cargando modelo desde {modelo_path}...")
        model = joblib.load(modelo_path)
    else:
        print(f"Entrenando modelo para {industry_type} - {column_name}...")
        model = Prophet(
            changepoint_prior_scale=cps,
            seasonality_prior_scale=sps,
            seasonality_mode=sm,
            yearly_seasonality=True,
            weekly_seasonality=False,
        )
        model.fit(df_prophet)
        # Guardar el modelo entrenado
        joblib.dump(model, modelo_path)
        print(f"Modelo guardado en {modelo_path}.")
    
    return model


In [ ]:
# Función para cargar los datos y filtrar la serie de tiempo seleccionada
def cargar_y_preparar_datos(df, industry_type, column_name):
    """
    Filtra y prepara los datos para Prophet según el tipo de industria y la columna seleccionada.
    """
    df_filtered = df[df['industry'] == industry_type][['date', column_name]].copy()
    df_filtered.columns = ['ds', 'y']  # Renombrar columnas para Prophet
    df_filtered['ds'] = pd.to_datetime(df_filtered['ds'])  # Asegurar formato de fecha
    return df_filtered if not df_filtered['y'].isnull().all() else None

In [ ]:
def pronóstico_con_grid_search(df_prophet, df_params, industry_type, column_name, periodos, frecuencia):
    """
    Realiza el pronóstico utilizando un modelo Prophet guardado o entrenado.
    """
    # Obtener los mejores parámetros
    filtered_df = df_params[(df_params['industry'] == industry_type) & (df_params['column'] == column_name)]
    
    if not filtered_df.empty:
        cps = filtered_df['changepoint_prior_scale'].iloc[0]
        sps = filtered_df['seasonality_prior_scale'].iloc[0]
        sm = filtered_df['seasonality_mode'].iloc[0]
    else:
        print("No se encontraron datos para la combinación especificada.")
        return

    # Cargar o entrenar modelo
    model = cargar_o_entrenar_modelo(df_prophet, industry_type, column_name, cps, sps, sm)

    # Realizar predicción
    future = model.make_future_dataframe(periods=periodos, freq=frecuencia)
    forecast = model.predict(future)

    # Guardar datos del pronóstico
    guardar_datos_forecast(forecast, industry_type, column_name)

    # Graficar predicción
    graficar_predicción(df_prophet, column_name, forecast, industry_type)
    

In [ ]:
""""Selector de opciones embebido en Jupyter Notebook"""

# Cargar los parámetros
df_params = pd.read_csv(r"mejores_modelos.csv")

# Crear un selector para la industria
industry_types = df_params['industry'].unique()
industry_selector = widgets.Dropdown(
    options=industry_types,
    description='Industria:',
    disabled=False,
)

# Función para actualizar las columnas en función de la industria seleccionada
def update_columns(industry):
    columns = df_params[df_params['industry'] == industry]['column'].unique()
    column_selector.options = columns

# Conectar el cambio de valor del selector de industria
industry_selector.observe(lambda change: update_columns(change['new']), names='value')

# Crear un selector para las columnas
column_selector = widgets.Dropdown(
    options = df_params[df_params['industry'] == industry_types[0]]['column'].unique(),
    description='Columna:',
    disabled=False,
)


periodo_selector = widgets.IntSlider(value=128, min=1, max=128, step=1, description='Períodos:')
display(industry_selector, column_selector, periodo_selector)

Dropdown(description='Industria:', options=('FHV - High Volume', 'FHV - Other', 'Green Taxi', 'Total Mercado',…

Dropdown(description='Columna:', options=('total_trips', 'unique_vehicles', 'total_amount', 'total_co2_emissio…

IntSlider(value=128, description='Períodos:', max=128, min=1)

Elegir las opciones de arriba y solo correr el script de abajo.

In [ ]:
periodos = periodo_selector.value
frecuencia = "MS"

industry_type = industry_selector.value
column_name = column_selector.value


# Cargar los parámetros
df_params = pd.read_csv(r"mejores_modelos.csv")
# Preparar los datos
df_prophet = cargar_y_preparar_datos(df, industry_type, column_name)


if df_prophet is not None and not df_prophet.empty and df_prophet['y'].notnull().all():

    #graficar_original(df_prophet, column_name)

    # Ejecutar Grid Search
    pronóstico_con_grid_search(df_prophet, df_params, industry_type, column_name, periodos=periodos, frecuencia=frecuencia)

Cargando modelo desde models/prophet_model_FHV - Other_total_trips.pkl...
Pronóstico para la combinación 'FHV - Other' - 'total_trips' ya existe en ..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output.csv.
